In [1]:
!apt-get update && apt-get install -y ffmpeg

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1581 B]
Get:2 http://archive.ubuntu.com/ubuntu jammy InRelease [270 kB]                
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [945 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]      
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]        
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]      
Get:7 http://archive.ubuntu.com/ubuntu jammy/universe amd64 Packages [17.5 MB] 
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2944 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 Packages [27.8 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy/main amd64 Packages [1792 kB]    
Get:12 http://archive.ubuntu.com/ubuntu jam

In [2]:
!pip3 install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 3.1 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 22.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 MB 35.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.5/776.5 kB 136.4 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801358 sha256=06f5e300de100e95899f6d2a2987912e586d4d537b218997f2c89f0e0d409a6a
  Stored in directory: /root/

In [3]:
!pip3 install pytubefix

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 926.5 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [4]:
import whisper
model = whisper.load_model("large")

100%|█████████████████████████████████████| 2.88G/2.88G [00:37<00:00, 82.6MiB/s]


In [5]:
import pytubefix as pt
yt = pt.YouTube("https://www.youtube.com/watch?v=Z07Ewop7rQA")
stream = yt.streams.filter(only_audio=True)[0]
stream.download(filename="aip.mp3")

'/workspace/repos/agentic-ai/aip.mp3'

In [6]:

result = model.transcribe("aip.mp3")

In [7]:
import json
with open("aip.json", "w") as f:
    json.dump(result, f)

In [8]:
list(result.keys())

['text', 'segments', 'language']

In [12]:
print(result['text'])

 Welcome to the AIP Asset Management and XIPO webinar. As a reminder, all participants are in listen-only mode and the meeting is being recorded. If you are on a mobile device, it's recommended that you turn the device to a horizontal position for a larger view of the slide deck. After the presentation, there will be an opportunity to ask questions. At any time during the meeting, you may click on the Q&A icon on the left-hand side of the screen to submit your question in writing. I would now like to turn the meeting over to Jay Bala with XIPO. Please go ahead. Good morning, everyone, and thank you for taking the time to join us today. My name is Jay Bala. I am the president and CEO of Access Pre-IPOs. It's a company that's giving investors exposure and access to global large-cap assets. We are a multi-talented, high-growth private company that is going to be a big shareholder in the market and is going to be a key part of the company's growth. This will be a company that will hold the

In [15]:
# !cp '/workspace/repos/agentic-ai/PPM - MCG MADISON RIDGE DST.pdf' /workspace/data
# !cp '/workspace/repos/agentic-ai/MASTER - PYTHON - SCORING MODEL - MCG MADISON RIDGE DST - v2.0.xlsx' /workspace/data
!apt update
!apt install tmux vim -y
!pip3 install llama-index==0.10.67post1 llama-parse llama-index-llms-huggingface llama-index-embeddings-huggingface llama-index-llms-ollama llama-index-finetuning
!pip3 install openpyxl sentencepiece protobuf evaluate rouge_score absl-py tensorboardX bitsandbytes peft accelerate python-dotenv graspologic
!pip3 install flash-attn --no-build-isolation
!curl -L https://ollama.com/download/ollama-linux-amd64 -o /usr/bin/ollama
!chmod +x /usr/bin/ollama
!useradd -r -s /bin/false -m -d /usr/share/ollama ollama

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease               
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InReleasem                
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease               
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease   
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
83 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tmux is already the newest version (3.2a-4ubuntu0.2).
vim is already the newest version (2:8.2.3995-1ubuntu2.17).
0 upgraded, 0 newly installed, 0 to remove and 83 not upgraded.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice]

In [16]:
!ollama pull llama3.1:latest

In [ ]:
import os
import torch
from llama_index.embeddings.adapter.base import LinearAdapterEmbeddingModel
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

from llama_index.core.embeddings import resolve_embed_model
from transformers import AutoTokenizer
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
    PropertyGraphIndex,
    Settings,
    Document
)


from dotenv import load_dotenv
load_dotenv('/workspace/repos/agentic-ai/.env')

import nest_asyncio
nest_asyncio.apply()

access_token = os.getenv('HF_TOKEN')
llama_api_key = os.getenv('LLAMA_API_KEY')

Settings.llm = llm

Settings.chunk_size = 500
Settings.chunk_overlap = 50
embed_model_name = "Alibaba-NLP/gte-Qwen2-1.5B-instruct"
# embed_model_name = "BAAI/bge-small-en-v1.5"
# embed_model_name = "hkunlp/instructor-base"
print("loading embed model...")
finetune_path = "/workspace/data/adapter_model_fullFT"
embed_model = HuggingFaceEmbedding(model_name=embed_model_name)

Settings.embed_model = embed_model


In [ ]:
documents = [Document(text=t, 
                          text_template='{metadata_str}\n\n{content}',
                          metadata=get_metadata(m, t, metadata={"section":None, "description":None, "mentioned_sections":None})) \
                            for m,t in data.items()]

In [ ]:
from llama_index.core import VectorStoreIndex, Settings
vector_index = VectorStoreIndex.from_documents(documents, embed_model=embed_model, show_progress=True)
vector_index.storage_context.persist(persist_dir=persist_dir)